# Path

Figure out which team was played for first
* Futbin is lower player_id
* Understat is earlier first date in matches or second team in comma list

In [1]:
import pandas as pd

futbin = pd.read_csv("futbin_2023.csv")

In [4]:
# Fix name column to match understat output
futbin = futbin.rename(columns={"name": "player_name"})
futbin

,player_name,pos,card_id,year,link,player_id,dob,rating,pac,acceleration,...,jumping,stamina,strength,aggression,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning
0,Matias Viña,LWB,51930,23,/23/player/51930/matias-vina,253290,09-11-1997,75,81.0,82.0,...,77.0,81.0,65.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kamaldeen Sulemana,LW,51890,23,/23/player/51890/kamaldeen-sulemana,256035,15-02-2002,75,93.0,92.0,...,77.0,70.0,60.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Albert Sambi Lokonga,CM,51875,23,/23/player/51875/albert-sambi-lokonga,241928,22-10-1999,75,66.0,63.0,...,71.0,79.0,63.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Cédric,RB,51870,23,/23/player/51870/cedric,201118,31-08-1991,75,71.0,76.0,...,72.0,66.0,57.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Anthony Gordon,LM,51855,23,/23/player/51855/anthony-gordon,242964,24-02-2001,75,80.0,78.0,...,58.0,75.0,56.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,Thomas Müller,CAM,26415,23,/23/player/26415/thomas-muller,189596,13-09-1989,87,69.0,66.0,...,77.0,86.0,66.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN
1341,João Cancelo,LB,51852,23,/23/player/51852/joao-cancelo,210514,27-05-1994,88,85.0,86.0,...,72.0,91.0,62.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN
1342,Joshua Kimmich,CDM,26274,23,/23/player/26274/joshua-kimmich,212622,08-02-1995,89,68.0,77.0,...,79.0,94.0,68.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN
1343,Sadio Mané,LM,26273,23,/23/player/26273/sadio-mane,208722,10-04-1992,89,90.0,91.0,...,87.0,85.0,72.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
db_id_dict = {}


names = futbin.player_name
dobs = futbin.dob

for i in range(len(names)):
    cur = names[i]
    if cur in db_id_dict:
        db_id_dict[cur] += 1
    else:
        db_id_dict[cur] = 1

In [8]:
db_id_dict.values()

dict_values([2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 

In [9]:
db_id_dict

{'Matias Viña': 2,
 'Kamaldeen Sulemana': 2,
 'Albert Sambi Lokonga': 2,
 'Cédric': 2,
 'Anthony Gordon': 2,
 'Ayew': 1,
 'Mario Lemina': 2,
 'Georginio Rutter': 2,
 'Maximilian Wöber': 1,
 'Ainsley Maitland-Niles': 1,
 'Layvin Kurzawa': 1,
 'Armando Broja': 1,
 'Luke Thomas': 1,
 'Brennan Johnson': 1,
 'Aaron Hickey': 1,
 'Bryan Mbeumo': 1,
 'Tariq Lamptey': 1,
 'Junior Firpo': 1,
 'Ethan Pinnock': 1,
 'Joseph Willock': 1,
 'Morgan Gibbs-White': 1,
 'Ryan Sessegnon': 1,
 'Yoane Wissa': 1,
 'Orel Mangala': 1,
 'Joe Aribo': 1,
 'Tom Davies': 1,
 'Dominic Solanke': 1,
 'Mason Holgate': 1,
 'Rico Henry': 1,
 'Kristoffer Ajer': 1,
 'Joelinton': 1,
 'Marvelous Nakamba': 1,
 'Marek Rodák': 1,
 'Tosin Adarabioyo': 1,
 'Philip Billing': 1,
 'Jefferson Lerma': 1,
 'Alex Iwobi': 1,
 'Danny Ward': 1,
 'Robin Olsen': 1,
 'Solly March': 1,
 'Will Hughes': 1,
 'Sam Johnstone': 1,
 'Harrison Reed': 1,
 'Calum Chambers': 1,
 'Ryan Fredericks': 1,
 'Stuart Dallas': 1,
 'Jamaal Lascelles': 1,
 'Jannik V